# Basic Operations

## Substitution

In the prior section, you learned how to build the sympy expressions.

In [1]:
from sympy import *

x = Symbol('x')
f = x**2 + x + 1
f

x**2 + x + 1

However, how to edit the mathematical expression?

You are not able to edit by re-assigning the variable 

In [2]:
x = Symbol('y')
f

x**2 + x + 1

And you are not able to edit the sympy expression directly,
because sympy protects it.

In [3]:
f.args[1] = Symbol('y')

TypeError: 'tuple' object does not support item assignment

In fact, the only ways to make a new formula is:

Destroying the old formula and try to build new formula by scratch.


In [4]:
y = Symbol('y')
y**2 + y + 1

y**2 + y + 1

However, in this way, you can't reuse the old formula programmatically.

The programatic way to replace $x$ by $y$ is substitution

In [5]:
x = Symbol('x')
y = Symbol('y')
g = f.subs(x, y)
g

y**2 + y + 1

Because sympy expressions are immutable, the substitution doesn't change the original expression

In [6]:
f

x**2 + x + 1

It's similar to string `replace` in Python

In [7]:
f = "x**2 + x + 1"
g = f.replace('x', '(y + 1)')
g

'(y + 1)**2 + (y + 1) + 1'

In [8]:
f

'x**2 + x + 1'

Substitution the variable with general function works

(with automatic evaluation)

In [9]:
x = Symbol('x')
y = Symbol('y')
expr = x**2 + x + 1
expr.subs(x, y + 1)

y + (y + 1)**2 + 2

You can also substitute multiple things at once using a dictionary.

In [10]:
expr.subs({x: 0, y: 1})

1

### Exercises

- Substitute $x \to 1$ to $x^2 + 2x - 3$ to check the solution
- Substitute $x \to \frac{-b + \sqrt{b^2 - 4ac}}{2a}$ to $x^2 + 1$. Then substitute a=1, b=0, c=1 into this expression to check the solution.

## Numeric Evaluation

### Evaluating $\pi$

In Python math library, you can only get 17 digits of $\pi$

In [11]:
import math

math.pi

3.141592653589793

How do you get 100 digits of $\pi$? or even 1000 digits?

In [12]:
pi.evalf(100)

3.141592653589793238462643383279502884197169399375105820974944592307816406286208998628034825342117068

### Evaluation by substitution

Since substitution also have evaluation semantics, you can use `subs` with numeric values for convenience.

In fact, if sympy sees that every arguments of functions are floating points, it considers the function as 'numeric' and immediately evaluates to a number.

In [13]:
(sin(x) / x).subs(x, 3.0)

0.0470400026866224

### Exact Arithmetic

Unless asked for numeric evaluation, SymPy tries to keep the formula as mathematically precise as possible.

For example, square roots and logarithms are irrational, and if you start to express it into some number, 
you lose some information how the number come from.

For example, take an example of a golden ratio

In [14]:
expr = (1 + sqrt(5)) / 2

In [15]:
expr

1/2 + sqrt(5)/2

If you evaluate it first to number

In [16]:
expr.evalf()

1.61803398874989

Now given that number, can you guess the original number back?

It is not possible because there can be so many plausible guesses, like below.

In [17]:
display(Rational(34, 21).evalf())
display(Rational(233, 144).evalf())
display(Rational(610, 377).evalf())

1.61904761904762

1.61805555555556

1.61803713527851

This example shows that sympy numbers satisfies the mathematical identities exactly,
while `math` library always has precision issues.

In [18]:
sqrt(2) ** 2 - 2

0

In [19]:
math.sqrt(2) ** 2 - 2

4.440892098500626e-16

The math library or numpy library is insufficient to use in actual mathematics, than numeric analysis.

So if you actually want to do exact math, SymPy is the only option.

Every SymPy functions, can be used to construct arbitrary mathematical constant that can be generated by mathematical formula.

In [20]:
sin(1)

sin(1)

For example, Apery's constant (https://en.wikipedia.org/wiki/Ap%C3%A9ry%27s_constant) is not available in standard math library. 

But if you need it, you can build it your own by using sympy functions.

In [21]:
zeta(3).evalf()

1.20205690315959

### Exercises

- Expand the decimals of $\frac{100}{9899}$ in 10, 20, 30, 40 digits
- Expand the decimals of $\frac{1000}{9801}$ in 10, 20, 30, 40 digits

## Parsing

We have learned how to build the mathematical formula by programming with symbols.

However, you can alternatively build SymPy expressions by parsing from strings.

The pros of using parser is that:

- It is easy to store expression as file and read to SymPy (serializing/deserializing).
- It can use more 'mathematical' grammar for expressions than Python:
  - You always need to declare symbols first, and undefined program variables don't become symbols
  - You can't multiply by juxtaposition

### SymPy parser

`parse_expr` can be used to parse text into SymPy expressions.

`parse_expr` can do the following:

- Parses `sin, cos, ...` into sympy functions.
- Finds undefined variables in the code like `x, y` and tries to inject symbols with same name

In [22]:
from sympy import *

parse_expr('sin(x + y)')

sin(x + y)

You can notice that factorial can be used directly
(with standard_transformations option)

And with `implicit_multiplication` option, you can multiply by juxtaposition.

In [23]:
from sympy.parsing.sympy_parser import standard_transformations, implicit_multiplication

parse_expr('2x**2 + 3x + 4', transformations=standard_transformations + (implicit_multiplication,))

2*x**2 + 3*x + 4

However, the cons is that even though parsing allows you to create symbols without declaring,

you would anyway need symbols later for computing derivatives, solving, ...
(to be introduced)

For that cases, you have to parse strings again for symbols:

In [24]:
parse_expr('sin(x + y)').diff(parse_expr('x'))

cos(x + y)

### Exercises

- Parse `3 x y - y x**3 - x**2 + 2 x + x y**2 + x**2 y + y**2 x**2`
- Parse `1 - sin(2*x)**2/4 - sin(y)**2 - cos(x)**4`

## Printing

### Math printers

Although you mostly see the LaTeX typesetted expression here,

It is SymPy's `latex` function that does real work behind

(The magic is done by `_repr_latex_` interface from IPython)

Calling `latex` is useful if you want to copy the typesetted SymPy expressions into your LaTeX article.

In [25]:
latex(exp(sin(x**2)))

'e^{\\sin{\\left(x^{2} \\right)}}'

(Be careful that Python escapes the backslashes if string is represented)

In [26]:
print(latex(exp(sin(x**2))))

e^{\sin{\left(x^{2} \right)}}


And you can use `pprint` (pretty printing) if you still need to typeset the mathematical expressions
with the ascii art (like the graphing calculator).

(However, you would only need `latex` for most of the cases)

In [27]:
pprint(exp(sin(x**2)))

    ⎛ 2⎞
 sin⎝x ⎠
ℯ       


In [28]:
pprint(exp(sin(x**2)), use_unicode=False)

    / 2\
 sin\x /
e       


And if you want to print with plaintext just use `str`:

In [29]:
str(exp(sin(x**2)))

'exp(sin(x**2))'

### Exercise

- Try to print `sin(pi/60)` in LaTeX and copy-paste into a Markdown cell

**EDIT THIS CELL**

<!-- Paste the result here between the dollar signs-->

$$  $$

# Advanced Topic: Pattern Matching

The pattern matching is useful for finding the coffecients for mathematical formula.

For example, if you want to check if the formula is a quadraitic equation,
you can define some pattern expression $ax^2 + bx + c$ using `Wild`.

In [30]:
x = Symbol('x')
a = Wild('a')
b = Wild('b')
c = Wild('c')

pattern = a*x**2 + b*x + c
pattern

x**2*a_ + x*b_ + c_

And you can use `expr.match(pattern)` to find the coefficients for the substitution.

In [31]:
expr = 2*x**2 + 2*x + 3
match = expr.match(pattern)
match

{a_: 2, b_: 2, c_: 3}

It gives a substitution as solution satifying the equation

`pattern.subs(match) == expr`

In [32]:
pattern.subs(match)

2*x**2 + 2*x + 3

We'd note that because of some ambiguity of pattern matching,
it can give some undesired (but correct) answer.

For example, the following match is correct syntactically, however, you wouldn't expect $x$ to appear in the coefficient

In [33]:
expr = x**2 + 2*x + 3
match = expr.match(pattern)
match

{a_: 2/x, b_: x, c_: 3}

In [34]:
pattern.subs(match)

x**2 + 2*x + 3

For this case, you would need to use `exclude` option.

In [35]:
a = Wild('a', exclude=[x])
b = Wild('b', exclude=[x])
c = Wild('c', exclude=[x])
pattern = a*x**2 + b*x + c

match = expr.match(pattern)
match

{b_: 2, a_: 1, c_: 3}

### Exercises

- Define a pattern of differential binomial $x^m (a + bx^n)^p$
- Match the pattern with $x$, $2x$, $x + 1$, $2x + 1$, $x^2 (x + 1)^3$